Predict the relation between premise and claim for the micro label corpus

In [ ]:
from transformers import TextClassificationPipeline
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import pickle
import torch
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:
#load data
claim_type_frame=pickle.load( open("match_bert_frame_micro.p", "rb"))

In [3]:
claim_type_frame


,Text_prem,Text_sup,Label,text
18,And many a video game is physically and partic...,It would be fair to make them into an Olympic ...,0,And many a video game is physically and partic...
6,That 's why it makes sense that the bachelor t...,The idea of the bachelor degree is to put toge...,1,That 's why it makes sense that the bachelor t...
2,Large events that stir national or even intern...,Hence the Mayor of Berlin ought to keep the hu...,0,Large events that stir national or even intern...
4,"However , no serious economist has confirmed t...",Excessive consumption of meat is unhealthy,1,"However , no serious economist has confirmed t..."
6,but this issue should be solvable through a cl...,Video games ought to be included in the ranks ...,0,but this issue should be solvable through a cl...
...,...,...,...,...
7,"However it 's mainly temps ( that is , student...",so this has hardly any effect on the level of ...,0,"However it 's mainly temps ( that is , student..."
7,I think Fritz has never been in a fight in his...,but that hardly counts as a proper brawl,1,I think Fritz has never been in a fight in his...
8,the EU should intervene right away,Humanity has made enough mistakes throughout h...,1,the EU should intervene right away Humanity ha...
34,as they can adjust their expenses through the ...,Rent increases upon re-letting aren't bad as s...,1,as they can adjust their expenses through the ...


In [4]:
 #load model

possible_labels = claim_type_frame.Label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

 model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


model.load_state_dict(torch.load('finetuned_BERT_epoch_5_claim_patch.model', map_location=torch.device('cpu')))


#load tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=None)

text_list = claim_type_frame['text'].tolist()

label_list=[]
for j in text_list:
        #predict label
        res=pipe(j)
        
        hold = [item for sublist in res for item in sublist]
        result = {}
        for k in hold:
            result.update(dict([tuple(k.values())]))

        label_list.append(max(result, key=result.get))

    


In [6]:
#add label to dataframe
claim_type_frame["pred_label"]=label_list

claim_type_frame["pred_label"].replace("LABEL_0","0",inplace=True)
claim_type_frame["pred_label"].replace("LABEL_1","1",inplace=True)

claim_type_frame


,Text_prem,Text_sup,Label,text,pred_label
18,And many a video game is physically and partic...,It would be fair to make them into an Olympic ...,0,And many a video game is physically and partic...,0
6,That 's why it makes sense that the bachelor t...,The idea of the bachelor degree is to put toge...,1,That 's why it makes sense that the bachelor t...,1
2,Large events that stir national or even intern...,Hence the Mayor of Berlin ought to keep the hu...,0,Large events that stir national or even intern...,0
4,"However , no serious economist has confirmed t...",Excessive consumption of meat is unhealthy,1,"However , no serious economist has confirmed t...",0
6,but this issue should be solvable through a cl...,Video games ought to be included in the ranks ...,0,but this issue should be solvable through a cl...,0
...,...,...,...,...,...
7,"However it 's mainly temps ( that is , student...",so this has hardly any effect on the level of ...,0,"However it 's mainly temps ( that is , student...",0
7,I think Fritz has never been in a fight in his...,but that hardly counts as a proper brawl,1,I think Fritz has never been in a fight in his...,0
8,the EU should intervene right away,Humanity has made enough mistakes throughout h...,1,the EU should intervene right away Humanity ha...,1
34,as they can adjust their expenses through the ...,Rent increases upon re-letting aren't bad as s...,1,as they can adjust their expenses through the ...,0


In [7]:
claim_type_frame.groupby(["Label","pred_label"]).count()

Text_prem  Text_sup  text
Label pred_label                           
0     0                 397       397   397
      1                  67        67    67
1     0                 366       366   366
      1                  98        98    98

In [20]:

y_gold= claim_type_frame['Label'].tolist()
y_pred=claim_type_frame["pred_label"].tolist()
y_pred=list(map(int, y_pred))

print("acc",accuracy_score(y_gold,y_pred))
print("f1",f1_score(y_gold,y_pred,average="macro"))


<class 'int'>
acc 0.5334051724137931
f1 0.47935624391830345
